In [1]:
import os
import pandas as pd
import numpy as np
import re

import sys
module_path = re.sub(r'Notebooks','Python Scripts',os.getcwd())
sys.path.append(module_path)
from performance_helper import *

2023-06-12 23:11:00.467897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def main(year, month, file, gap_length):
    if not re.search(r'\d{4}',year):
        raise Exception(f"Incorret Input: {year}")
    elif not re.search(r'[A-Za-z]{3}',month):
        raise Exception(f"Incorret Input: {month}")
    elif not [file_i for file_i in ['Irradiance','Deger','Fixed'] if re.search(fr'{file}',file_i)]:
        raise Exception(f"Incorret Input: File")
    else:
        path_list,_ = get_file_paths(file)
        path_list = path_function(year,month,path_list)
        df = df_cleaner(path_list,file)
    return df, file, year, month, gap_length

In [3]:
def imputation_methods(test_df, df_copy, ml_df, test_gaps):
    imputation_dict = {}
#     zoo_functions(test_df.copy())
    imputation_dict['MLP_Multi'] = mlp_multi(test_df.copy(), ml_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['MLP'] = mlp(test_df.copy(), ml_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['Kalman Smoothing'] = kalman(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
    imputation_dict['ARIMA'] = ARIMA(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['Seasonal Decomposition'] = seasonal_decom(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['Zoo Interpolation'] = zoo_interpolation(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['Zoo Spline'] = zoo_spline(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
#     imputation_dict['Forecast Interpolation'] = forecast_interpolate(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
    imputation_dict['Python Interpolate'] = interpolate(test_df.copy(), df_copy.copy(), test_gaps).to_dict()
    imputation_df = pd.DataFrame.from_dict({(outerKey, innerKey): values for outerKey, innerDict in imputation_dict.items() for innerKey, values in innerDict.items()}).T
    return imputation_df

In [4]:
def performance_test(df, file, gap_length):
    
    imputation_df = pd.DataFrame()
    
    for _ in range(1):
                
        test_df = df.copy()
    
        test_df, base_df = missing_value_simulation(test_df, gap_length)
        
        test_df, base_df, month_i = missing_data_beg_end_month(test_df, base_df, file)
        
        test_df, ml_df = meteoblue_data(test_df, file)

        test_gaps = map_nan_gaps_indexes(test_df, month_i)
        
        output_df = imputation_methods(test_df, base_df, ml_df, test_gaps)
        
        if imputation_df.empty:
            imputation_df = output_df.copy()
        else:
            imputation_df = (imputation_df + output_df) / 2
            
    return imputation_df, test_gaps

In [5]:
def performance(imputation_df):    
    r2_df = pd.DataFrame(imputation_df.drop(['mae','mse','rmse'], axis=0,level=1).max(axis=0),columns = ['r2'])
    r2_df['Method'] = imputation_df.drop(['mae','mse','rmse'], axis=0,level=1).idxmax(axis=0).values
    mae_df = pd.DataFrame(imputation_df.drop(['r2','mse','rmse'], axis=0,level=1).min(axis=0),columns = ['mae'])
    mae_df['Method'] = imputation_df.drop(['r2','mse','rmse'], axis=0,level=1).idxmin(axis=0).values
    rmse_df = pd.DataFrame(imputation_df.drop(['r2','mse','mae'], axis=0,level=1).min(axis=0),columns = ['rmse'])
    rmse_df['Method'] = imputation_df.drop(['r2','mse','mae'], axis=0,level=1).idxmin(axis=0).values
    mse_df = pd.DataFrame(imputation_df.drop(['r2','rmse','mae'], axis=0,level=1).min(axis=0),columns = ['mse'])
    mse_df['Method'] = imputation_df.drop(['r2','rmse','mae'], axis=0,level=1).idxmin(axis=0).values
    for row in r2_df.index:
        print(row)
        print(f"Optimal Imputation Method for {row}: {r2_df.loc[row]['Method'][0]}, R2 score: {round(r2_df.loc[row]['r2'],5)}")
        print(f"Optimal Imputation Method for {row}: {mae_df.loc[row]['Method'][0]}, MAE score: {round(mae_df.loc[row]['mae'],5)}")
        print(f"Optimal Imputation Method for {row}: {rmse_df.loc[row]['Method'][0]}, RMSE score: {round(rmse_df.loc[row]['rmse'],5)}")
        print(f"Optimal Imputation Method for {row}: {mse_df.loc[row]['Method'][0]}, MSE score: {round(mse_df.loc[row]['mse'],5)}\n")
        
    return imputation_df

In [6]:
# = Load all the Data = #
test_df, file, year, month, gap_length = main(year = input("Year (format: YYYY): "),month = input("Month (format: jul): "),
     file = input("File (opt: Irradiance/Deger/Fixed): "), gap_length = int(input("Gap size (seconds): ")))

# = perform imputation test = #
imputation_df, test_gaps = performance_test(test_df, file, gap_length)

Year (format: YYYY): 2022
Month (format: jul): jul
File (opt: Irradiance/Deger/Fixed): Irradiance
Gap size (seconds): 200


2023-06-12 23:12:12.065261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 93ms/step


In [7]:
## = print imputation test results = #
print(f"\nImputation methods for {file} and error statistics for {year}, {month} with a {gap_length} second gap from {test_df.index[min(test_gaps[test_df.columns[0]])]} - {test_df.index[max(test_gaps[test_df.columns[0]])]}.\n")
performance(imputation_df)


Imputation methods for Irradiance and error statistics for 2022, jul with a 200 second gap from 2022-07-25 11:56:20+00:00 - 2022-07-25 11:59:00+00:00.

GlobalIR
Optimal Imputation Method for GlobalIR: Python Interpolate, R2 score: -0.28654
Optimal Imputation Method for GlobalIR: Python Interpolate, MAE score: 98.61435
Optimal Imputation Method for GlobalIR: Python Interpolate, RMSE score: 111.88745
Optimal Imputation Method for GlobalIR: Python Interpolate, MSE score: 12518.80063

DirectIR
Optimal Imputation Method for DirectIR: ARIMA, R2 score: 0.3459
Optimal Imputation Method for DirectIR: ARIMA, MAE score: 7.65951
Optimal Imputation Method for DirectIR: ARIMA, RMSE score: 8.76159
Optimal Imputation Method for DirectIR: ARIMA, MSE score: 76.76547

DiffuseIR
Optimal Imputation Method for DiffuseIR: Python Interpolate, R2 score: -0.58824
Optimal Imputation Method for DiffuseIR: Python Interpolate, MAE score: 102.77917
Optimal Imputation Method for DiffuseIR: Python Interpolate, RMSE s

GlobalIR      DirectIR     DiffuseIR  WindSpeed  \
MLP_Multi          mae      411.827581    186.064157    219.165889   0.616483   
                   mse   179107.978894  34773.026892  56766.239325   0.467572   
                   rmse     423.211506    186.475272    238.256667   0.683792   
                   r2       -17.406742   -295.293252     -5.361879  -3.714220   
ARIMA              mae      116.122656      7.659511    109.731290   0.303983   
                   mse    18112.642323     76.765465  16388.532322   0.128295   
                   rmse     134.583217      8.761590    128.017703   0.358184   
                   r2        -0.861418      0.345898     -0.836688  -0.293519   
Python Interpolate mae       98.614352      8.412963    102.779167   0.320926   
                   mse    12518.800627     91.234290  14171.657726   0.127686   
                   rmse     111.887446      9.551664    119.044772   0.357332   
                   r2        -0.286544      0.222613     -0.588239  -0.287375   

                         Temperature  
MLP_Multi          mae     10.313761  
                   mse    106.464280  
                   rmse    10.318153  
                   r2   -3476.260754  
ARIMA              mae      0.045155  
                   mse      0.002821  
                   rmse     0.053110  
                   r2       0.907874  
Python Interpolate mae      0.086111  
                   mse      0.008866  
                   rmse     0.094158  
                   r2       0.710433